In [5]:
# x: patch embedded images + cls_token, shape (batch_size x num_patches + cls_token x embed_dim) (8, 2561, 768)
# B: batch_size (8)
# T: time_dimension (32)
# W: num_img_patches_width (10)

# def forward(self, x, B, T, W): 
#     num_spatial_tokens = (x.size(1) - 1) // T
#     H = num_spatial_tokens // W   # (8)
    
#     if self.attention_type in ['space_only', 'joint_space_time']:
#             x = x + self.drop_path(self.attn(self.norm1(x)))
#             x = x + self.drop_path(self.mlp(self.norm2(x)))
#             return x

#     elif self.attention_type == 'divided_space_time':
#         ## Temporal
#         xt = x[:,1:,:]  # (8, 2560, 768) taking all patches except the one to classify
#         xt = rearrange(xt, 'b (h w t) m -> (b h w) t m',b=B,h=H,w=W,t=T)  # (640, 32, 768) ??? NO TRANSPOSE ??? rearrange such that it is being iterated over dim T
#         res_temporal = self.drop_path(self.temporal_attn(self.temporal_norm1(xt)))  # (640, 32, 768) calculate attention
#         res_temporal = rearrange(res_temporal, '(b h w) t m -> b (h w t) m',b=B,h=H,w=W,t=T)  # (8, 2560, 768) rearrange to shape from before
#         res_temporal = self.temporal_fc(res_temporal)  # (8, 2560, 768)
#         xt = x[:,1:,:] + res_temporal   # (8, 2560, 768) add temporal attention to x

#         ## Spatial
#         init_cls_token = x[:,0,:].unsqueeze(1)  # (8, 1, 768) take out cls_token 
#         cls_token = init_cls_token.repeat(1, T, 1)  # (8, 32, 768) repeat cls_token T times
#         cls_token = rearrange(cls_token, 'b t m -> (b t) m',b=B,t=T).unsqueeze(1)  # (256, 1, 768)
#         xs = xt  # (8, 2560, 768)
#         xs = rearrange(xs, 'b (h w t) m -> (b t) (h w) m',b=B,h=H,w=W,t=T)  # (256, 80, 768) rearrange such that it is being iterated over space (h x w)
#         xs = torch.cat((cls_token, xs), 1)  # (256, 81, 768) add patch to classify to xs
#         res_spatial = self.drop_path(self.attn(self.norm1(xs)))  # (256, 81, 768) calculate attention

#         ### Taking care of CLS token
#         cls_token = res_spatial[:,0,:]  # (256, 768) taking out patch to classify
#         cls_token = rearrange(cls_token, '(b t) m -> b t m',b=B,t=T)  # (8, 32, 768) rearrange for temporal dimension
#         cls_token = torch.mean(cls_token,1,True) ## averaging for every frame (8, 1, 768) averaging over time dimension
#         res_spatial = res_spatial[:,1:,:]  # (256, 80, 768) taking all patches except the one to classify in time dimension shape
#         res_spatial = rearrange(res_spatial, '(b t) (h w) m -> b (h w t) m',b=B,h=H,w=W,t=T)  # (8, 2560, 768) rearrange to batch size x num patches
#         res = res_spatial  # (8, 2560, 768)
#         x = xt  # (8, 2560, 768)

#         ## Mlp
#         x = torch.cat((init_cls_token, x), 1) + torch.cat((cls_token, res), 1) # (8, 1, 768) , (8, 2560, 768)  +  (8, 1, 768) , (8, 2560, 768)
#         x = x + self.drop_path(self.mlp(self.norm2(x)))  # multi layer perceptron
#         return x  # (8, 2561, 768)

In [1]:
import numpy as np

In [102]:
orig_array = np.array([[[[1,2,3,4,5],
                [6,7,8,9,10],
                [11,12,13,14,15],
                [16,17,18,19,20]],

                [[100,200,300,400,500],
                [600,700,800,900,1000],
                [1100,1200,1300,1400,1500],
                [1600,1700,1800,1900,2000]],

                [[10000,20000,30000,40000,50000],
                [60000,70000,80000,90000,100000],
                [110000,120000,130000,140000,150000],
                [160000,170000,180000,190000,200000]]],

                [[[11,21,31,41,51],
                [61,71,81,91,101],
                [111,121,131,141,151],
                [161,171,181,191,201]],

                [[1001,2001,3001,4001,5001],
                [6001,7001,8001,9001,10001],
                [11001,12001,13001,14001,15001],
                [16001,17001,18001,19001,20001]],

                [[100001,200001,300001,400001,500001],
                [600001,700001,800001,900001,1000001],
                [1100001,1200001,1300001,1400001,1500001],
                [1600001,1700001,1800001,1900001,2000001]]]])

# Patch embedding
array = orig_array.reshape(2, 1, 3, 4, 5) # batch channel time height width
array = array.reshape(6,1,4,5)
array = array.reshape(6,1,20)
array = array.reshape(6,20,1)

# Put batch size to the front
embedded_array = array.reshape(2,60,1)
    
    
# x: patch embedded images + cls_token, shape (batch_size x num_patches + cls_token x embed_dim) (2, 60, 1)
# B: batch_size (2)
# T: time_dimension (3)
# W: num_img_patches_width (5)

In [3]:
# Rearranga array like done in divided space time attention (does not work as expected)
array = embedded_array.reshape(40, 3, 1)


In [103]:
# Rearrange for time: THIS is how it should look like
array = embedded_array.reshape(2, 3, 20, 1)
array

array([[[[      1],
         [      2],
         [      3],
         [      4],
         [      5],
         [      6],
         [      7],
         [      8],
         [      9],
         [     10],
         [     11],
         [     12],
         [     13],
         [     14],
         [     15],
         [     16],
         [     17],
         [     18],
         [     19],
         [     20]],

        [[    100],
         [    200],
         [    300],
         [    400],
         [    500],
         [    600],
         [    700],
         [    800],
         [    900],
         [   1000],
         [   1100],
         [   1200],
         [   1300],
         [   1400],
         [   1500],
         [   1600],
         [   1700],
         [   1800],
         [   1900],
         [   2000]],

        [[  10000],
         [  20000],
         [  30000],
         [  40000],
         [  50000],
         [  60000],
         [  70000],
         [  80000],
         [  90000],
         [ 10000

In [ ]:
t = array.transpose((0, 2, 1, 3))
t = t.reshape(40, 3, 1)
t

In [5]:
! pip install einops

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [26]:
# integrating changes in the code from timeSformer -> models -> vit.py -> Block
from einops import rearrange
B = 2
H = 4
W = 5
T = 3

xt = embedded_array
# xt = x[:,1:,:]
xt = rearrange(xt, 'b (h w t) m -> b t (h w) m',b=B,h=H,w=W,t=T) #xt.reshape(2, 3, 20, 1)
xt = xt.transpose(0,2,1,3) #((0, 2, 1, 3))
xt = rearrange(xt, 'b (h w) t m -> (b h w) t m',b=B,h=H,w=W,t=T)  #t.reshape(40, 3, 1)

# new approach:
# N = 4
# R = xt.shape[0] / N
# xt = rearrange(xt, '(x n) t m -> x n t m',b=B,h=H,w=W,t=T, n=N, x=R)
# # t = t.reshape(10, 4, 3, 1)
# xt = rearrange(xt, 'x n t m -> x (n t) m',b=B,h=H,w=W,t=T, n=N, x=R)  # reshape such that 4 patches in each layer are considered
# # t = t.reshape(10, 12, 1)

# attention
# res_temporal = self.drop_path(self.temporal_attn(self.temporal_norm1(xt)))
res_temporal = xt

# new approach
# xt = rearrange(xt, 'x (n t) m -> x n t m',b=B,h=H,w=W,t=T, n=N, x=R)
# xt = rearrange(xt, 'x n t m -> (x n) t m',b=B,h=H,w=W,t=T, n=N, x=R)

# corrected
res_temporal = rearrange(res_temporal, '(b h w) t m -> b (h w) t m',b=B,h=H,w=W,t=T)
res_temporal = res_temporal.transpose(0,2,1,3) #(0, 2, 1, 3))
res_temporal = rearrange(res_temporal, 'b t (h w) m -> b (h w t) m',b=B,h=H,w=W,t=T)

# res_temporal = self.temporal_fc(res_temporal)
# xt = x[:,1:,:] + res_temporal

In [27]:
res_temporal.shape

(2, 60, 1)

In [23]:
from einops import rearrange
B = 2
H = 4
W = 5
T = 3

xt = embedded_array
xt.shape

(2, 60, 1)

In [66]:
N = 4
R = int(xt.shape[0] / N)

xt = embedded_array
xt = rearrange(xt, 'b (h w t) m -> b t (h w) m',b=B,h=H,w=W,t=T) #xt.reshape(2, 3, 20, 1)
# xt = xt.transpose(0,2,1,3) #((0, 2, 1, 3))
# xt = rearrange(xt, 'b t (h w) m -> (b h w) t m',b=B,h=H,w=W,t=T)  #t.reshape(40, 3, 1)

# xt = rearrange(xt, '(x n) t m -> (x) n t m',t=T, n=N, x=R)
# # t = t.reshape(10, 4, 3, 1)
# xt = rearrange(xt, 'x n t m -> x (n t) m',t=T, n=N, x=R)  # reshape such that 4 patches in each layer are considered
# # t = t.reshape(10, 12, 1)

0


In [117]:
####### Solution
xt = embedded_array
# xt = rearrange(xt, 'b (t h w) m -> b (h w) t m',b=B,h=H,w=W,t=T)
xt = rearrange(xt, 'b (t h w) m -> (b h w) t m',b=B,h=H,w=W,t=T)
# xt.shape

In [118]:
####### Solution
xt = rearrange(xt, '(b h w) t m -> b (t h w) m',b=B,h=H,w=W,t=T)
xt

array([[[      1],
        [      2],
        [      3],
        [      4],
        [      5],
        [      6],
        [      7],
        [      8],
        [      9],
        [     10],
        [     11],
        [     12],
        [     13],
        [     14],
        [     15],
        [     16],
        [     17],
        [     18],
        [     19],
        [     20],
        [    100],
        [    200],
        [    300],
        [    400],
        [    500],
        [    600],
        [    700],
        [    800],
        [    900],
        [   1000],
        [   1100],
        [   1200],
        [   1300],
        [   1400],
        [   1500],
        [   1600],
        [   1700],
        [   1800],
        [   1900],
        [   2000],
        [  10000],
        [  20000],
        [  30000],
        [  40000],
        [  50000],
        [  60000],
        [  70000],
        [  80000],
        [  90000],
        [ 100000],
        [ 110000],
        [ 120000],
        [ 13

In [25]:
xt.shape

(2, 20, 3, 1)

In [27]:
xt = xt.transpose(0,2,1,3)

In [29]:
xt.shape

(2, 3, 20, 1)

In [87]:
orig_array = np.array([[[[1,2,3,4,5],
                [6,7,8,9,10],
                [11,12,13,14,15],
                [16,17,18,19,20]],

                [[100,200,300,400,500],
                [600,700,800,900,1000],
                [1100,1200,1300,1400,1500],
                [1600,1700,1800,1900,2000]],

                [[10000,20000,30000,40000,50000],
                [60000,70000,80000,90000,100000],
                [110000,120000,130000,140000,150000],
                [160000,170000,180000,190000,200000]]]])

# array = orig_array.reshape(1, 3, 4, 5) #channel time height width
array = array.reshape(3,4,5)
array = array.reshape(3,20)
array = array.reshape(3,20)

# Put batch size to the front
embedded_array = array.reshape(60)

In [88]:
from einops import rearrange
B = 2
H = 4
W = 5
T = 3

xt = embedded_array
xt.shape

(60,)

In [89]:
xt = rearrange(xt, '(h w t) -> t (h w)',h=H,w=W,t=T) #xt.reshape(2, 3, 20, 1)

In [90]:
xt.shape

(3, 20)

In [91]:
xt

array([[     1,      4,      7,     10,     13,     16,     19,    200,
           500,    800,   1100,   1400,   1700,   2000,  30000,  60000,
         90000, 120000, 150000, 180000],
       [     2,      5,      8,     11,     14,     17,     20,    300,
           600,    900,   1200,   1500,   1800,  10000,  40000,  70000,
        100000, 130000, 160000, 190000],
       [     3,      6,      9,     12,     15,     18,    100,    400,
           700,   1000,   1300,   1600,   1900,  20000,  50000,  80000,
        110000, 140000, 170000, 200000]])

In [92]:
xt = xt.transpose()

In [93]:
xt

array([[     1,      2,      3],
       [     4,      5,      6],
       [     7,      8,      9],
       [    10,     11,     12],
       [    13,     14,     15],
       [    16,     17,     18],
       [    19,     20,    100],
       [   200,    300,    400],
       [   500,    600,    700],
       [   800,    900,   1000],
       [  1100,   1200,   1300],
       [  1400,   1500,   1600],
       [  1700,   1800,   1900],
       [  2000,  10000,  20000],
       [ 30000,  40000,  50000],
       [ 60000,  70000,  80000],
       [ 90000, 100000, 110000],
       [120000, 130000, 140000],
       [150000, 160000, 170000],
       [180000, 190000, 200000]])